In [5]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
import os
import matplotlib.colors as mcolors
import xarray as xr

In [2]:
#ACQUISITION

#All coral data files are from MAREANO
gml_file_coral_reefs_trondelag = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml"
gdf_coral_reefs_trondelag = gpd.read_file(gml_file_coral_reefs_trondelag)

gml_file_coral_reefs_more_og_romsdal = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_15_More_og_Romsdal_25832_Korallrev_GML.gml"
gdf_coral_reefs_more_og_roms = gpd.read_file(gml_file_coral_reefs_more_og_romsdal)

gml_file_coral_reefs_nordland = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_18_Nordland_25833_Korallrev_GML.gml"
gdf_coral_reefs_nordland = gpd.read_file(gml_file_coral_reefs_nordland)

gml_file_coral_reefs_vestland = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_46_Vestland_25832_Korallrev_GML.gml"
gdf_coral_reefs_vestland = gpd.read_file(gml_file_coral_reefs_vestland)

gml_file_coral_reefs_troms = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_55_Troms_25833_Korallrev_GML.gml"
gdf_coral_reefs_troms = gpd.read_file(gml_file_coral_reefs_troms)

gml_file_coral_reefs_finnmark = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_56_Finnmark_25833_Korallrev_GML.gml"
gdf_coral_reefs_finnmark = gpd.read_file(gml_file_coral_reefs_finnmark)

gml_file_coral_reefs_norskehavet = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_63_Norskehavet_25832_Korallrev_GML.gml"
gdf_coral_reefs_norskehavet = gpd.read_file(gml_file_coral_reefs_norskehavet)

gml_file_coral_reefs_barentshavet = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_64_Barentshavet_vest_25833_Korallrev_GML.gml"
gdf_coral_reefs_barentshavet = gpd.read_file(gml_file_coral_reefs_barentshavet)


#Define file paths to merge
gml_files = {
    "Trondelag": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml",
    "More_og_Romsdal": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_15_More_og_Romsdal_25832_Korallrev_GML.gml",
    "Nordland": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_18_Nordland_25833_Korallrev_GML.gml",
    "Vestland": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_46_Vestland_25832_Korallrev_GML.gml",
    "Troms": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_55_Troms_25833_Korallrev_GML.gml",
    "Finnmark": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_56_Finnmark_25833_Korallrev_GML.gml",
    "Norskehavet": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_63_Norskehavet_25832_Korallrev_GML.gml",
    "Barentshavet": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_64_Barentshavet_vest_25833_Korallrev_GML.gml"
}

#Read all files into a list of GeoDataFrames
gdfs = [gpd.read_file(file).to_crs("EPSG:4326") for file in gml_files.values()]

#Combine
gdf_all_coral_files = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


In [3]:
#CLEANING
#remove any entries with nan values for longitude or latitude (none of the points used here had this)
gdf_all_coral_files = gdf_all_coral_files.dropna(subset=['lengdegrad', 'breddegrad'])

In [6]:
#Load the reprojected and standardized SINMOD dataset
SINMOD_features_reprojected = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid_standardized.nc')


In [21]:
#Load the original SINMOD data from SINMOD grid
SINMOD_features_original_grid = xr.open_dataset('/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc')

In [24]:
#TRANSFORMATION

#first we need to obtain the SINMOD crs (coordinate reference system)

def obtain_sinmod_crs(sinmod_data):
    
    grid_mapping = sinmod_data.variables['grid_mapping']
    horizontal_resolution = grid_mapping.attrs.get('horizontal_resolution')

    crs_sinmod = CRS.from_proj4(f"+proj=stere "
                                f"+lat_0={grid_mapping.attrs.get('latitude_of_projection_origin')} "
                                f"+lat_ts={grid_mapping.attrs.get('standard_parallel')} "
                                f"+lon_0={grid_mapping.attrs.get('straight_vertical_longitude_from_pole')} "
                                f"+x_0={grid_mapping.attrs.get('false_easting')} "
                                f"+y_0={grid_mapping.attrs.get('false_northing')} "
                                f"+a={grid_mapping.attrs.get('semi_major_axis')} "
                                f"+b={grid_mapping.attrs.get('semi_minor_axis')} "
                                f"+units=m +no_defs")

    print(f"SINMOD crs: {crs_sinmod}")
    return crs_sinmod

crs_sinmod = obtain_sinmod_crs(SINMOD_features_original_grid)

#Next, transform all coral points to SINMOD grid coordinates

def transform_coral_coordinates(coral_data, crs_sinmod):
    
    #define the crs for the coral data
    crs_wgs84 = CRS.from_epsg(4326)  #WGS84 (lat/lon coordinates)

    transformer_wgs84_to_sinmod = Transformer.from_crs(crs_wgs84, crs_sinmod, always_xy=True)

    #transform coral coordinates from WGS84 to SINMOD
    coral_x, coral_y = transformer_wgs84_to_sinmod.transform(coral_data['lengdegrad'], coral_data['breddegrad'])

    #save coral_x and coral_y as new columns
    coral_data.loc[:, 'x'] = coral_x
    coral_data.loc[:, 'y'] = coral_y

    return coral_data

transformed_coral_data = transform_coral_coordinates(gdf_all_coral_files, crs_sinmod)


SINMOD crs: +proj=stere +lat_0=90.0 +lat_ts=60.0 +lon_0=58.0 +x_0=2544800.0 +y_0=1918800.0 +a=6370000.0 +b=6370000.0 +units=m +no_defs +type=crs


In [1]:
#Remove coral points outside model area and drop coral locations that are Nan in the SINMOD dataset

def remove_coral_points_outside_area(transformed_coral_data, sinmod_data):
    xc = sinmod_data.variables['xc'][:]
    yc = sinmod_data.variables['yc'][:]
    xmin, xmax = xc.min(), xc.max()
    ymin, ymax = yc.min(), yc.max()

    coral_x = transformed_coral_data['x']
    coral_y = transformed_coral_data ['y']
    
    valid_indices = ((coral_x >= xmin) & (coral_x <= xmax) & (coral_y >= ymin) & (coral_y <= ymax))

    coral_points_inside_area = transformed_coral_data[valid_indices].copy()

    null_mask = sinmod_data['bottom_temperature_features'].isel(stat=0).isnull()
    
    filtered_coral_ds = coral_points_inside_area[
    ~coral_points_inside_area.apply(lambda row: null_mask.sel(x=row['x'], y=row['y'], method='nearest').item(), axis=1)
    ]

    
    
return filtered_coral_ds

coral_data = remove_coral_points_outside_area(transformed_coral_data, SINMOD_features_reprojected)

NameError: name 'coral_ds' is not defined

In [ ]:
#save to your chosen directory
output_directory = '/coral-mapping/processed_data/'
output_file = 'coral_data_processed.parquet' 
os.makedirs(output_directory, exist_ok=True)

coral_data.to_parquet(os.path.join(output_directory, output_file), index=False)

In [ ]:
#How to load coral data later:
#coral_filepath = '/output_directory/output_file'
#coral_data = pd.read_parquet(coral_filepath)

In [ ]:
#OPTIONAL
#plot all coral points on top of bathymetry to view their position in model area

fig, ax = plt.subplots(figsize=(10, 10))

#Plot the reprojected SINMOD data (bottom_temperature, for example)
SINMOD_features_reprojected['bottom_temperature_features'].isel(stat=0).plot(ax=ax, cmap='viridis', alpha=0.6)

#Plot coral locations
ax.scatter(coral_ds['x'], coral_ds['y'], color='red', s=10, label='Coral Locations')

ax.set_title('Coral Locations on Reprojected SINMOD Temperature Features')
plt.xlabel("xc (SINMOD grid coordinates)")
plt.ylabel("yc (SINMOD grid coordinates)")
plt.legend()
plt.show()

In [25]:
coral_file_sinmod_grid = '/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/midnor_total_coral_data_processed.parquet'
coral_ds_sinmod_grid = pd.read_parquet(coral_file_sinmod_grid)

In [26]:
coral_file_emod_grid = '/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/midnor_total_coral_data_filtered_UTM32N.parquet'
coral_ds_emod_grid = pd.read_parquet(coral_file_emod_grid)